In [1]:
# 모델과 weight값 저장후 재 로딩할수 있게 필요...
#구글 드라이브 저장
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
print('gauth:',gauth)
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)
print('drive:', drive)

def gcp_upload(file_name):
    uploaded = drive.CreateFile({'title': file_name}) #, "parents": [{"kind": "drive#fileLink","id": 'jukyellow@gmail.com'}]})
    uploaded.SetContentString(file_name)
    uploaded.SetContentFile('./' + file_name)
    uploaded.Upload()
    print('Uploaded file with ID {}'.format(uploaded.get('id')))

gauth: <pydrive.auth.GoogleAuth object at 0x7f002498c080>
drive: <pydrive.drive.GoogleDrive object at 0x7f0015c9ecf8>


In [0]:
#구글 드라이브에서 다운로드
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
import io
from io import BytesIO 
from googleapiclient.http import MediaIoBaseDownload


def gcp_download(file_name, key):
    #3. 모델 다운로드
    #https://drive.google.com/open?id=1TlvbayGRCjAI6bOZrUYMmv6g6b95rnRM
    request = drive_service.files().get_media(fileId=key)

    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()
      if status:
          print("Download %%%d%%." % int(status.progress() * 100))
      print("Download Complete!")
    downloaded.seek(0)

    with open('/tmp/' + file_name, 'wb') as f:
        f.write(downloaded.read())

In [3]:
file_name = 'gd_sample.txt'
gcp_download(file_name, '1FaEvvmYgEAh99e_ubbeZPqdV64_A4w5M')

Download %100%.
Download Complete!


In [4]:
ls -alrt /tmp/

total 5460
drwxr-xr-x 1 root root    4096 Feb 27 11:44 ../
drwxrwxrwt 1 root root    4096 Feb 27 11:47 ./
-rw-r--r-- 1 root root 5578948 Feb 27 11:47 gd_sample.txt


In [5]:
# https://wikidocs.net/31698
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

sparse_x_path = '/tmp/' + file_name

corpus = []
with open(sparse_x_path) as f:
    for line in f:
        corpus.append(line)
corpus = np.array(corpus)
print('corpus.shape:', corpus.shape)

tfidfv = TfidfVectorizer().fit(corpus)
print('type(tfidfv):', type(tfidfv))

tfidf = tfidfv.transform(corpus)
print('tfidf.shape:', tfidf[0])
print('tfidf.shape:', tfidf.shape)
print('type(tfidf):', type(tfidf))

#tfidf_arr = tfidf.toarray()
#print(tfidf_arr.vocabulary_)

corpus.shape: (242366,)
type(tfidfv): <class 'sklearn.feature_extraction.text.TfidfVectorizer'>
tfidf.shape:   (0, 56522)	0.536620773656817
  (0, 56287)	0.5961121038878314
  (0, 46553)	0.5972340453108668
tfidf.shape: (242366, 81089)
type(tfidf): <class 'scipy.sparse.csr.csr_matrix'>


In [6]:
!git clone https://github.com/jukyellow/clustering4docs

Cloning into 'clustering4docs'...
remote: Enumerating objects: 268, done.
remote: Total 268 (delta 0), reused 0 (delta 0), pack-reused 268
Receiving objects: 100% (268/268), 1.15 MiB | 24.61 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [7]:
ls -alrt

total 32
drwxr-xr-x 1 root root 4096 Feb  5 18:37 sample_data/
drwxr-xr-x 1 root root 4096 Feb 27 11:44 ../
drwxr-xr-x 1 root root 4096 Feb 27 11:47 .config/
-rw-r--r-- 1 root root 2610 Feb 27 11:47 adc.json
drwxr-xr-x 1 root root 4096 Feb 27 11:47 ./
drwxr-xr-x 5 root root 4096 Feb 27 11:47 clustering4docs/


In [8]:
import time
import sys
#sys.path.append('../')
sys.path.append('./clustering4docs/')
import soyclustering

print(soyclustering.__version__)
n_clusters = 101

0.1.0


In [9]:
from soyclustering import SphericalKMeans

kmeans = SphericalKMeans(
    n_clusters=n_clusters,
    init='k-means++',
    sparsity=None,
    max_iter=10,
    tol=0.0001,
    verbose = True
)

t = time.time()
labels = kmeans.fit_predict(tfidf)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

initialization_time=2.956792 sec, sparsity=3.91e-05
n_iter=1, changed=97695, inertia=218978.984, iter_time=1.226 sec, sparsity=0.0168
n_iter=2, changed=66277, inertia=198782.419, iter_time=1.164 sec, sparsity=0.0199
n_iter=3, changed=47326, inertia=192701.134, iter_time=1.218 sec, sparsity=0.0218
n_iter=4, changed=26636, inertia=190074.128, iter_time=1.169 sec, sparsity=0.0222
n_iter=5, changed=15515, inertia=188605.172, iter_time=1.183 sec, sparsity=0.0224
n_iter=6, changed=10146, inertia=187836.850, iter_time=1.146 sec, sparsity=0.0225
n_iter=7, changed=7357, inertia=187361.539, iter_time=1.242 sec, sparsity=0.0225
n_iter=8, changed=5014, inertia=187027.311, iter_time=1.174 sec, sparsity=0.0225
n_iter=9, changed=3058, inertia=186890.563, iter_time=1.175 sec, sparsity=0.0224
n_iter=10, changed=2682, inertia=186797.612, iter_time=1.166 sec, sparsity=0.0225
process time = 14.977 seconds


In [10]:
import sklearn
print(sklearn.__version__)

# scikit-learn KMeans use only Euclidean
from sklearn.preprocessing import normalize
x_ = normalize(tfidf)

0.22.1


In [11]:
from sklearn.cluster import KMeans

kmeans_sklearn = KMeans(
    n_clusters=n_clusters,
    init='k-means++',
    n_init=1,
    max_iter=10,
    tol=0.0001,
    verbose=1
)

t = time.time()
_label = kmeans_sklearn.fit_predict(x_)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

Initialization complete
Iteration  0, inertia 239396.447
Iteration  1, inertia 232662.733
Iteration  2, inertia 227262.628
Iteration  3, inertia 225448.582
Iteration  4, inertia 224885.962
Iteration  5, inertia 224739.357
Iteration  6, inertia 224472.849
Iteration  7, inertia 224070.986
Iteration  8, inertia 224023.769
Iteration  9, inertia 223991.009
process time = 19.328 seconds


In [15]:
#word2index를 생성하기만 하면되는데, 굳이 word2vec를 만들필요는없음..
# 파라메터값 지정
num_features   = 300  # 문자 벡터 차원 수 (300:wiki, data가 적으므로 100차원만)
min_word_count = 1   # 최소 문자 수 : 해당 출현빈도수 이하의 단어는 무시(*학습하는데 부담이 되지 않는다면 일반적으로 작게 설정)
num_workers    = 4    # 병렬 처리 스레드 수
context        = 3    # 문자열 창 크기 (평균 문자 길이를 고려?: HS품명 평균3)
downsampling   = 1e-3 # 문자 빈도수 Downsample : 빈도수가 큰값에 대해 평준화적용
sg_type        = 1    # 0:CBOW(skip-gram보다 빠름), 1:skip-gram(느리지만 성능이 더좋음?)

# 초기화 및 모델 학습
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.wrappers import FastText

model_name = '300features_1minwords_3context_sg1_sample'
#gd_mode_key = '1G-aMJt4k5np7C_usIFZnguAxuVGffnke' # https://drive.google.com/open?id=1G-aMJt4k5np7C_usIFZnguAxuVGffnke

def createWord2vec(new_sentence, model_name, IS_UPLOAD=False):
    start = time.time() # 시작시간
    print('IS_W2V_CREATE START!') 
    # 모델 생성
    model = word2vec.Word2Vec(new_sentence, 
                              workers=num_workers, 
                              size=num_features, 
                              min_count=min_word_count,
                              window=context,
                              sample=downsampling,
                              sg = sg_type) 
    print(model)

    # 학습이 완료 되면 필요없는 메모리를 unload 시킨다.
    model.init_sims(replace=True)
    model.save(model_name)

    if IS_UPLOAD:
        # 2. predict 모델 저장/업로드
        #w2v model feature
        upload_weight('', model_name, drive)
        print('IS_W2V_CREATE END!')  

    end = time.time()
    elapsed = end - start
    print("Time taken for model creating: ", elapsed, "seconds.")
    return model

new_sentence = [words.split() for words in corpus]
print(len(new_sentence))
model = createWord2vec(new_sentence, model_name, IS_UPLOAD=False)

242366
IS_W2V_CREATE START!
Word2Vec(vocab=113846, size=300, alpha=0.025)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Time taken for model creating:  35.932371377944946 seconds.


In [16]:
from soyclustering import proportion_keywords

print('model.wv.index2word:', len(model.wv.index2word))

keywords = proportion_keywords(
    kmeans.cluster_centers_,
    labels,
    index2word=model.wv.index2word,
    topk=30,
    candidates_topk=100
)

model.wv.index2word: 113846


In [17]:
keywords[59]

[('MACRALEX', 1.0),
 ('OXIDIDED', 1.0),
 ('4/110', 1.0),
 ('4-3-3', 1.0),
 ('HWALSAM', 0.9995604962847932),
 ('KL-400', 0.9995589760633384),
 ('SIROKOSHI', 0.999445321747579),
 ('CME', 0.999402334117458),
 ('SALTRAM', 0.9993833679060897),
 ('PINNACLE', 0.998953657624274),
 ('LA-CE', 0.9989529316734753),
 ('KATUSO', 0.9987510308187098),
 ('BERMUDAGRASS', 0.9987004614136981),
 ('TOPMIX', 0.9981172561156201),
 ('OLDPOX', 0.9980843081729648),
 ('TONNA', 0.9963533048710835),
 ('CHAVE', 0.996308803461794),
 ('TARE,2.05KG,6PC/CTN(2016.06.01)', 0.9960643339293401),
 ('DAL(BLACK', 0.9960249229366923),
 ('REFF', 0.9959155614050198),
 ('BRUSCHETTA', 0.9950910498716886),
 ('RONAFLAIR', 0.9948179580502908),
 ('ENVIRONMENTAL', 0.9947687095102826),
 ('CURRY(BAMOND)-MILD', 0.9945306515214383),
 ('TT(G)', 0.9934679526871377),
 ('KANAMYCINE', 0.9931831872930224),
 ('BONEIN', 0.9931208311128459),
 ('0.25MOL/L-SULFURIC', 0.9921949154173182),
 ('DESACETYL', 0.992170722340245),
 ('SS', 0.9907668873976888)]

In [18]:
for cluster_idx, keyword in enumerate(keywords):
    if cluster_idx % 5 == 0:
        print()
    keyword = '///'.join([w for w,_ in keyword])
    print('cluster#{} : {}'.format(cluster_idx, keyword))


cluster#0 : 4-BROMO-2-METHOXYBENZYL///MEAT(B.Q.F)///LIVINGSTONII///MANDJI///190OTS///D.C.P///JAERAESIC///TRICHLORFON///BALBI///PUREWHITE///2-CHLOROPHENYLACETYLENE,///MXS///PTB///DUBAI///NONGSIMNEOGURI///PHOSPHIRC///OLIGOMERS///GROCERY;///PRESERVATIVE///DIDISCUS///POMIDORO///GYERANGWAJA///40MESH///HD(E)///TRUFFLESHELL///CRISIS///FISH(WITH///EPD///SPRD///VEAN
cluster#1 : HMC///B/IN///FLUSHINGS///EDULIS)SLICE///CHROMIUN///PARCE///ABSORB.///SOLUTION(N/40)///SULPHOSUCCINATE).///LYCOPODII///NO.23///SI2H6-4N8-10KG-44LDISS632///FLAKES(NON-GMO)///PEAR&WET///CYANUS///TOMT///VALSARTAN///monitor///DARMERA///GAFFEL///BAHIA///GINGER)///BUKID///HAL///JUICE(DRINK)///NITRATES,///REMOVAL///SAUCE(20%)(280G*24)///GUM(BOTTLE///BEEFTRMG(85.00CL)
cluster#2 : ALUMINM///RODENT///CASONA///513///DRINK(WON///PARTHENIUM///BORUYNG///RADIO///M/E&G/E-SYS///SSF///RAMARIA///SUREFLAM///REHYD///N,N,N,N-TETRAMETHYIETHYIENEDIAMINE///FROZ.RIB.F.W.ROUND///CT20///BIOCERA///AGLNCD///POSTERIOR///CHO-MATE///SOIUM///1,5-DIAMINOP